In [69]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import multiprocessing as mp
import json
from scipy.optimize import minimize,basinhopping
from functools import partial
from tqdm import tqdm

from mpl import discount
from mpl import choice_rule

In [70]:
discount.trade_d({'beta':1,'zeta':1,'alpha':1,'k':1},ss_t=10,ll_t=100)

1.168519262604985

In [76]:
c = choice_rule.utility(ss_x=5,ss_t=10,ll_x=100,ll_t=30,params=[1,1,1,1,2],dstyle='trade')

In [107]:
choice_rule.choice_prob(ss_x=5,ss_t=10,ll_x=100,ll_t=30,params=[.5,.1,.1],dstyle='attention',ustyle='power',temper=1)

0.7595604498955438

In [ ]:
cross_valid.test_model(style=style,test_sample=sample["test"],params=train_result["params"])


{'mse': 0.17418814587999581,
 'mae': 0.3508533636842815,
 'accuracy': 0.6583865351131747,
 'log_like': -4533.529694310184}

In [2]:
chavez_dt = pd.read_csv('data/chavez_data.csv')

# questionnaire design
pregunta = np.array([54,0,55,117,55,0,75,61,19,0,25,53,31,0,85,7,
                     14,0,25,19,47,0,50,160,15,0,35,13,25,0,60,14,
                     78,0,80,162,40,0,55,62,11,0,30,7,67,0,75,119,
                     34,0,35,186,27,0,50,21,69,0,85,91,49,0,60,89,
                     80,0,85,157,24,0,35,29,33,0,80,14,28,0,30,179,
                     34,0,50,30,25,0,30,80,41,0,75,20,54,0,60,111,
                     54,0,80,30,22,0,25,136,20,0,55,7]).reshape(27, 4)

condition_col = np.array(['DT'+str(i) for i in range(1, 28)]).reshape(-1,1)
pregunta = np.hstack((condition_col,pregunta))
pregunta = pd.DataFrame(pregunta)
condition_col_name = ['ss_amount','ss_delay','ll_amount','ll_delay']
pregunta.columns = ['condition'] + condition_col_name

# object containing participants' choices: '0' is choice of SS, '1' is choice of 'LL'
chavez_dt = pd.melt(chavez_dt, id_vars=['ID','School'], var_name='condition', value_name='choice')
chavez_dt = pd.merge(chavez_dt,pregunta,on='condition')
chavez_dt[condition_col_name] = chavez_dt[condition_col_name].apply(pd.to_numeric)
chavez_dt.columns = ['person_id','school','condition','choice'] + condition_col_name

chavez_dt

,person_id,school,condition,choice,ss_amount,ss_delay,ll_amount,ll_delay
0,1,1,DT1,0.0,54,0,55,117
1,2,1,DT1,0.0,54,0,55,117
2,3,1,DT1,0.0,54,0,55,117
3,4,1,DT1,1.0,54,0,55,117
4,5,1,DT1,0.0,54,0,55,117
...,...,...,...,...,...,...,...,...
34663,1552,8,DT27,1.0,20,0,55,7
34664,1553,8,DT27,1.0,20,0,55,7
34665,1554,8,DT27,1.0,20,0,55,7
34666,1555,8,DT27,1.0,20,0,55,7


In [2]:
itch_dt = pd.read_csv('data/itch_data.csv')
itch_dt = itch_dt.rename(columns={"Subject":"person_id",
                        "X1":"ss_amount",
                        "T1":"ss_delay",
                        "X2":"ll_amount",
                        "T2":"ll_delay",
                        "LaterOptionChosen": "choice"})
itch_dt

,person_id,Condition,Question,G,R,D,ss_amount,ss_delay,ll_amount,ll_delay,choice
0,259,4,1,0.5,1.50,1,1.00,2,1.50,3,0.0
1,259,4,2,2.0,4.00,1,0.67,0,2.67,1,0.0
2,259,4,3,10.0,1.01,1,1000.00,2,1010.00,3,0.0
3,259,4,4,0.5,4.00,2,0.17,0,0.67,2,0.0
4,259,4,5,10.0,1.50,2,20.00,0,30.00,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...
23495,793,1,21,10.0,1.25,3,40.00,2,50.00,5,1.0
23496,793,1,22,100.0,2.00,1,100.00,0,200.00,1,1.0
23497,793,1,23,500.0,2.00,3,500.00,0,1000.00,3,1.0
23498,793,1,24,10.0,1.01,2,1000.00,2,1010.00,4,0.0


In [24]:
dstyle_list = list(mpl_estimation.config_param['discount_func'].keys())
#ustyle_list = list(mpl_estimation.config_param['utility_func'].keys())
ustyle_list = ["power"]
style_list = [{"dstyle":dstyle_list[i],"ustyle":ustyle_list[j]} 
              for i in range(len(dstyle_list)) for j in range(len(ustyle_list))]

with mp.Pool(processes=4) as pool:
    result = []

    func = partial(cross_valid.validation,data=itch_dt,sample_times=20)
    for r in tqdm(pool.imap(func,style_list),total=len(style_list)):
        result += r

pool.join()
pool.close()

100%|██████████| 9/9 [48:46<00:00, 325.22s/it]   


In [25]:
with open('crossvalid_output.json', 'w') as f:
    json.dump(result, f)
sucess_result = [d for d in result if 'Fail to converge' not in d.values()]
sucess_result_tab = pd.DataFrame(sucess_result).groupby(['model']).mean()
sucess_result_tab['iter'] = pd.DataFrame(sucess_result).groupby(['model'])['iter'].count()
sucess_result_tab.to_csv("cross_valid_result_2.csv")

In [26]:
sucess_result_tab

,mse,mae,accuracy,log_like,iter
model,,,,,
attention-power,0.208908,0.417425,0.682846,-3515.803651,19
expo-power,0.205660,0.415521,0.691343,-3485.903809,20
expo2-power,0.205394,0.415116,0.690721,-3490.879764,14
hb-power,0.207483,0.416106,0.685180,-3505.014120,19
hb2-power,0.209631,0.406708,0.680632,-3566.917307,20
hbmd-power,0.202267,0.405576,0.700174,-3434.337778,20
hce-power,0.208268,0.415015,0.683770,-3515.479530,10
quasihb-power,0.207830,0.382915,0.681961,-3864.065715,20
quasihb_fc-power,0.208875,0.395679,0.677600,-3815.240673,20


In [96]:
data = chavez_dt
init_params = [.9,1,.01,1]
dstyle = 'attention'
ustyle = 'power'
bounds = [(.01,1),(.01,100),(.01,100),(.01,100)]
result = minimize(mpl_estimation.log_likelihood, 
         x0=init_params, args=(data,dstyle,ustyle,bounds), 
         bounds=bounds,
         tol=1e-8,
         method='L-BFGS-B')
result

      fun: 16974.212425537
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.16516424, -0.02874003,  0.06984919,  0.00691216])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 220
      nit: 26
     njev: 44
   status: 0
  success: True
        x: array([0.98793912, 1.17045354, 0.42731155, 1.57849465])

In [4]:
data = chavez_dt
x0 = [.5,1,.5,1]
bounds = [(.01,1),(.01,100),(.01,100),(.01,100)]
dstyle = 'attention'
ustyle = 'power'
minimizer_kwargs = {"method": "L-BFGS-B", "args": (data, dstyle, ustyle,bounds)}

solver = basinhopping(mpl_estimation.log_likelihood, x0, minimizer_kwargs=minimizer_kwargs, 
             niter=1000, 
             stepsize=0.05, 
             T=1.0, 
             niter_success = 100,
             disp=False)

e:\Attention_discounting\attention_discount_project\model\discount_func.py:88: RuntimeWarning: overflow encountered in power
  g = 1 / (1 - dargs['delta']) * (dargs['delta']**(-t) - 1)
e:\Attention_discounting\attention_discount_project\model\mpl_estimation.py:38: RuntimeWarning: divide by zero encountered in log
  log_like = np.sum(np.log(p_choice))
d:\Python\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [72]:
data = itch_dt
style = {'dstyle':'attention','ustyle':'power','method':'probit'}
mpl_estimation.mle(style,data,disp_output=True,disp_step=True)

basinhopping step 0: f 14871.3
basinhopping step 1: f 14871.3 trial_f 1e+10 accepted 0  lowest_f 14871.3
basinhopping step 2: f 14871.3 trial_f 14874.8 accepted 0  lowest_f 14871.3
basinhopping step 3: f 14871.5 trial_f 14871.5 accepted 1  lowest_f 14871.3


KeyboardInterrupt: 

In [69]:
params = [1,.5,10]
bounds = [[.001,100],[.001,100],[.001,100]]
dstyle = 'hbmd'
ustyle = 'power'
data = itch_dt
simu_size = 1000
kwargs = {'simu_sample':np.random.normal(size=len(data)*simu_size).reshape(len(data),simu_size)}

In [70]:
mpl_estimation.log_likelihood(params, data, dstyle, ustyle, bounds, method="probit", regenerate_sample=False,kwargs=kwargs)

15978.410516612446

In [27]:


dstyle_list = list(mpl_estimation.config_param['discount_func'].keys())
ustyle_list = list(mpl_estimation.config_param['utility_func'].keys())
style_list = [{"dstyle":dstyle_list[i],"ustyle":ustyle_list[j]} 
              for i in range(len(dstyle_list)) for j in range(len(ustyle_list))]

with mp.Pool(processes=4) as pool:
    result = []

    func = partial(mpl_estimation.mle,data=itch_dt)
    for r in tqdm(pool.imap(func,style_list),total=len(style_list)):
        result += [r]

pool.join()
pool.close()

100%|██████████| 27/27 [08:10<00:00, 18.17s/it]


In [28]:
success_result = list(np.array(result)[[result[i] != "Fail to converge" for i in range(len(result))]])
pd.DataFrame(success_result)

,model,params,se,gradient,log-likelihood,aic,bic
0,expo-power,"[0.994, 0.013, 0.011]","[0.001, 0.003, 0.002]","[-2.43, 0.698, -2.146]",-14202.379,28410.759,28465.147
1,expo-cara,"[0.743, 2.045, 0.509]","[0.001, 0.015, 0.002]","[0.0, -0.0, 0.002]",-15165.959,30337.919,30392.307
2,hb-power,"[0.011, 0.025, 0.021]","[0.001, 0.002, 0.002]","[-1.314, 0.111, 0.699]",-14192.353,28390.705,28445.094
3,hb-cara,"[0.295, 2.936, 0.398]","[0.0, 0.008, 0.0]","[-0.0, 0.0, 0.001]",-15175.382,30356.764,30411.153
4,hb2-power,"[0.399, 0.004, 0.002, 0.002]","[0.007, 0.007, 0.003, 0.002]","[-19.79, 1860.486, -464.905, -4091.861]",-14044.266,28096.531,28169.049
5,hb2-cara,"[0.036, 8.124, 2.146, 0.489]","[0.009, 0.007, 0.014, 0.042]","[0.422, 0.002, 0.001, -0.02]",-15165.669,30339.338,30411.856
6,hbmd-power,"[7.094, 0.146, 0.171]","[0.021, 0.001, 0.0]","[0.0, 0.0, -0.001]",-13928.870,27863.740,27918.129
7,hbmd-cara,"[3.453, 2.461, 0.389]","[0.002, 0.003, 0.0]","[0.0, 0.0, -0.001]",-15155.590,30317.179,30371.568
8,quasihb-power,"[0.999, 0.999, 0.001, 0.001]","[0.004, 0.003, 0.009, 0.007]","[-3271.168, -2270.061, 464.412, -3364.671]",-14062.506,28133.012,28205.530
9,quasihb-cara,"[0.943, 0.784, 2.294, 0.465]","[0.003, 0.003, 0.042, 0.003]","[-0.001, -0.001, 0.0, 0.001]",-15162.805,30333.610,30406.128


In [38]:
pd.DataFrame(success_result).to_csv("mle_result_3.csv",index=False)

In [54]:
style = {"dstyle":"attention_uni","ustyle":"log"}
res = mpl_estimation.mle(style=style,data=itch_dt)

e:\Attention_discounting\attention_discount_project\model\mpl_estimation.py:39: RuntimeWarning: divide by zero encountered in log
  p_choice_ss = 1 - p_choice_ll
d:\Python\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [56]:
res = pd.DataFrame({'model': 'attention_uni-log',
 'params': [[1.295, 1.505, 0.384, 0.282]],
 'se': [[0.004, 0.005, 0.001, 0.001]],
 'gradient': [[0.0, 0.0, -0.0, 0.001]],
 'log-likelihood': -13992.422,
 'aic': 27992.844,
 'bic': 28065.363})

In [40]:
style = {"dstyle":"attention_uni","ustyle":"power"}
res = mpl_estimation.mle(style=style,data=itch_dt)

e:\Attention_discounting\attention_discount_project\model\mpl_estimation.py:39: RuntimeWarning: divide by zero encountered in log
  p_choice_ss = 1 - p_choice_ll
d:\Python\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [68]:
tot_res = pd.read_csv('mle_result_3.csv')
#pd.concat([tot_res,res]).to_csv('mle_result_3.csv')

In [69]:
impo

,model,params,se,gradient,log-likelihood,aic,bic
0,expo-power,"[0.994, 0.013, 0.011]","[0.001, 0.003, 0.002]","[-2.43, 0.698, -2.146]",-14202.379,28410.759,28465.147
1,expo-cara,"[0.743, 2.045, 0.509]","[0.001, 0.015, 0.002]","[0.0, -0.0, 0.002]",-15165.959,30337.919,30392.307
2,hb-power,"[0.011, 0.025, 0.021]","[0.001, 0.002, 0.002]","[-1.314, 0.111, 0.699]",-14192.353,28390.705,28445.094
3,hb-cara,"[0.295, 2.936, 0.398]","[0.0, 0.008, 0.0]","[-0.0, 0.0, 0.001]",-15175.382,30356.764,30411.153
4,hb2-power,"[0.399, 0.004, 0.002, 0.002]","[0.007, 0.007, 0.003, 0.002]","[-19.79, 1860.486, -464.905, -4091.861]",-14044.266,28096.531,28169.049
5,hb2-cara,"[0.036, 8.124, 2.146, 0.489]","[0.009, 0.007, 0.014, 0.042]","[0.422, 0.002, 0.001, -0.02]",-15165.669,30339.338,30411.856
6,hbmd-power,"[7.094, 0.146, 0.171]","[0.021, 0.001, 0.0]","[0.0, 0.0, -0.001]",-13928.870,27863.740,27918.129
7,hbmd-cara,"[3.453, 2.461, 0.389]","[0.002, 0.003, 0.0]","[0.0, 0.0, -0.001]",-15155.590,30317.179,30371.568
8,quasihb-power,"[0.999, 0.999, 0.001, 0.001]","[0.004, 0.003, 0.009, 0.007]","[-3271.168, -2270.061, 464.412, -3364.671]",-14062.506,28133.012,28205.530
9,quasihb-cara,"[0.943, 0.784, 2.294, 0.465]","[0.003, 0.003, 0.042, 0.003]","[-0.001, -0.001, 0.0, 0.001]",-15162.805,30333.610,30406.128


In [26]:
test_result = cross_valid.validation(style=style,data=itch_dt,sample_times=20)

  5%|▌         | 1/20 [00:15<04:45, 15.04s/it]

{'mse': 0.2043127151235341, 'mae': 0.40907320981355777, 'accuracy': 0.6928013876843018, 'log_like': -3447.1841697758045}


 10%|█         | 2/20 [00:26<03:51, 12.87s/it]

{'mse': 0.20451927004690942, 'mae': 0.41079338822248906, 'accuracy': 0.6974818903070024, 'log_like': -3470.6954145232503}


 15%|█▌        | 3/20 [00:34<03:02, 10.71s/it]

...

{'mse': 0.20044607415268254, 'mae': 0.40474951631903067, 'accuracy': 0.7048473348283595, 'log_like': -3411.21442431893}


 95%|█████████▌| 19/20 [04:19<00:14, 14.34s/it]

{'mse': 0.20185773703125223, 'mae': 0.4076368546083672, 'accuracy': 0.7014718614718615, 'log_like': -3423.6531836622557}


100%|██████████| 20/20 [04:26<00:00, 13.33s/it]

{'mse': 0.19873558603805372, 'mae': 0.405722621518874, 'accuracy': 0.7043010752688172, 'log_like': -3382.0717694721725}


In [27]:
m = pd.DataFrame(test_result).groupby(['model']).mean().reset_index()
m['iter'] = len(test_result)

In [28]:
m

,model,mse,mae,accuracy,log_like,iter
0,attention_uni-log,0.202205,0.407536,0.700601,-3431.676035,20


In [37]:
sucess_res = pd.read_csv('cross_valid_result_2.csv')
#pd.concat([sucess_res,m]).to_csv('cross_valid_result_2.csv',index=False)
sucess_res

,model,mse,mae,accuracy,log_like,iter
0,attention-power,0.208908,0.417425,0.682846,-3515.803651,19
1,expo-power,0.205660,0.415521,0.691343,-3485.903809,20
2,expo2-power,0.205394,0.415116,0.690721,-3490.879764,14
3,hb-power,0.207483,0.416106,0.685180,-3505.014120,19
4,hb2-power,0.209631,0.406708,0.680632,-3566.917307,20
5,hbmd-power,0.202267,0.405576,0.700174,-3434.337778,20
6,hce-power,0.208268,0.415015,0.683770,-3515.479530,10
7,quasihb-power,0.207830,0.382915,0.681961,-3864.065715,20
8,quasihb_fc-power,0.208875,0.395679,0.677600,-3815.240673,20
9,attention_uni-power,0.206063,0.414933,0.691464,-3478.057401,20
